In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['CLS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [25]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [26]:
list(zip(longforms, counts))

[('chronological lifespan', 110),
 ('chronological life span', 66),
 ('crown like structures', 62),
 ('capillary leak syndrome', 18),
 ('coffin lowry syndrome', 15),
 ('capillary like structures', 10),
 ('cardiolipin synthase', 9),
 ('centrosomal localization signal', 8),
 ('cyst like structures', 7),
 ('clitoral stimulation', 6),
 ('closantel', 6),
 ('catheter lock solution', 4),
 ('contact lens sensor', 4),
 ('cl synthase', 4),
 ('condomless sex', 4),
 ('capture long seq', 3),
 ('circular leaf spot', 3),
 ('cre like sequence', 3),
 ('ciliary localization signal', 3),
 ('chloramphenicol succinate', 3),
 ('cilostazol', 3),
 ('camp response element like sequence', 3),
 ('chemical lumbar sympathectomy', 2),
 ('chikui likelihood scale', 2),
 ('closed loop system', 2),
 ('conventional laparoscopic surgery', 2),
 ('cercospora leaf spot', 2),
 ('cytoplasmic localization signal', 2)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-03 01:49:41] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'camp response element like sequence': 'ungrounded',
  'capillary leak syndrome': 'MESH:D019559',
  'capillary like structures': 'capillary_like_structures',
  'capture long seq': 'ungrounded',
  'cardiolipin synthase': 'HGNC:16148',
  'catheter lock solution': 'ungrounded',
  'centrosomal localization signal': 'centrosome_localization_signal',
  'chloramphenicol succinate': 'CHEBI:CHEBI:3606',
  'chronological life span': 'chronological_lifespan',
  'chronological lifespan': 'chronological_lifespan',
  'ciliary localization signal': 'ungrounded',
  'cilostazol': 'CHEBI:CHEBI:31401',
  'circular leaf spot': 'ungrounded',
  'cl synthase': 'ungrounded',
  'clitoral stimulation': 'ungrounded',
  'closantel': 'CHEBI:CHEBI:77053',
  'coffin lowry syndrome': 'MESH:D038921',
  'condomless sex': 'ungrounded',
  'contact lens sensor': 'ungrounded',
  'cre like sequence': 'ungrounded',
  'crown like structures': 'NCIT:C116010',
  'cyst like structures': 'ungrounded'},
 {'MESH:D019559': 'Capill

In [27]:
grounding_map, names, pos_labels = [{'camp response element like sequence': 'ungrounded',
  'capillary leak syndrome': 'MESH:D019559',
  'capillary like structures': 'capillary_like_structures',
  'capture long seq': 'ungrounded',
  'cardiolipin synthase': 'HGNC:16148',
  'catheter lock solution': 'ungrounded',
  'centrosomal localization signal': 'centrosome_localization_signal',
  'chloramphenicol succinate': 'CHEBI:CHEBI:3606',
  'chronological life span': 'chronological_lifespan',
  'chronological lifespan': 'chronological_lifespan',
  'ciliary localization signal': 'ungrounded',
  'cilostazol': 'CHEBI:CHEBI:31401',
  'circular leaf spot': 'ungrounded',
  'cl synthase': 'HGNC:16148',
  'clitoral stimulation': 'ungrounded',
  'closantel': 'CHEBI:CHEBI:77053',
  'coffin lowry syndrome': 'MESH:D038921',
  'condomless sex': 'ungrounded',
  'contact lens sensor': 'ungrounded',
  'cre like sequence': 'ungrounded',
  'crown like structures': 'NCIT:C116010',
  'cyst like structures': 'ungrounded'},
 {'MESH:D019559': 'Capillary Leak Syndrome',
  'capillary_like_structures': 'capillary_like_structures',
  'HGNC:16148': 'CRLS1',
  'centrosome_localization_signal': 'centrosome_localization_signal',
  'CHEBI:CHEBI:3606': 'Chloramphenicol succinate',
  'chronological_lifespan': 'chronological_lifespan',
  'CHEBI:CHEBI:31401': 'cilostazol',
  'CHEBI:CHEBI:77053': 'closantel',
  'MESH:D038921': 'Coffin-Lowry Syndrome',
  'NCIT:C116010': 'Crown-Like Structure'},
 ['HGNC:16148', 'MESH:D019559', 'MESH:D038921', 'NCIT:C116010']]

In [28]:
excluded_longforms = []

In [29]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [30]:
additional_entities = {'HGNC:16148': ['CRLS1', ['CLS', 'CLS1']]}

In [31]:
unambiguous_agent_texts = {}

In [32]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [33]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [34]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [35]:
intersection1

[('HGNC:16148', 'HGNC:16148', 22)]

In [36]:
intersection2

[('CHEBI:CHEBI:77053', 'HGNC:16148', 0),
 ('ungrounded', 'HGNC:16148', 0),
 ('MESH:D019559', 'HGNC:16148', 0),
 ('MESH:D038921', 'HGNC:16148', 0),
 ('centrosome_localization_signal', 'HGNC:16148', 0),
 ('chronological_lifespan', 'HGNC:16148', 0),
 ('capillary_like_structures', 'HGNC:16148', 0),
 ('CHEBI:CHEBI:3606', 'HGNC:16148', 0),
 ('HGNC:16148', 'HGNC:16148', 1),
 ('NCIT:C116010', 'HGNC:16148', 0),
 ('CHEBI:CHEBI:31401', 'HGNC:16148', 0)]

In [37]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [38]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [39]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 02:55:31] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 02:55:36] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9320357142857144 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [40]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:77053': 4,
  'ungrounded': 31,
  'MESH:D019559': 14,
  'MESH:D038921': 13,
  'centrosome_localization_signal': 5,
  'chronological_lifespan': 113,
  'capillary_like_structures': 6,
  'CHEBI:CHEBI:3606': 1,
  'HGNC:16148': 14,
  'NCIT:C116010': 36,
  'CHEBI:CHEBI:31401': 2},
 'f1': {'mean': 0.932036, 'std': 0.046694},
 'precision': {'mean': 0.9775, 'std': 0.027839},
 'recall': {'mean': 0.910833, 'std': 0.063114},
 'HGNC:16148': {'f1': {'mean': 0.92, 'std': 0.09798},
  'pr': {'mean': 0.866667, 'std': 0.163299},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D019559': {'f1': {'mean': 0.764762, 'std': 0.170197},
  'pr': {'mean': 0.733333, 'std': 0.249444},
  'rc': {'mean': 0.883333, 'std': 0.145297}},
 'CHEBI:CHEBI:3606': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.92028, 'std': 0.005594},
  'pr': {'mean': 0.938095, 'std': 0.07619},
  'rc': {'mean': 0.914286, 'std'

In [41]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [42]:
disamb.dump(model_name, results_path)

In [43]:
print(disamb.info())

Disambiguation model for CLS

Produces the disambiguations:
	CRLS1*	HGNC:16148
	Capillary Leak Syndrome*	MESH:D019559
	Chloramphenicol succinate	CHEBI:CHEBI:3606
	Coffin-Lowry Syndrome*	MESH:D038921
	Crown-Like Structure*	NCIT:C116010
	capillary_like_structures	capillary_like_structures
	centrosome_localization_signal	centrosome_localization_signal
	chronological_lifespan	chronological_lifespan
	cilostazol	CHEBI:CHEBI:31401
	closantel	CHEBI:CHEBI:77053

Class level metrics:
--------------------
Grounding                     	Count	F1     
        chronological_lifespan	113	0.96185
          Crown-Like Structure*	 36	    1.0
                    Ungrounded	 31	0.92028
       Capillary Leak Syndrome*	 14	0.76476
                         CRLS1*	 14	   0.92
         Coffin-Lowry Syndrome*	 13	   0.96
     capillary_like_structures	  6	0.73333
centrosome_localization_signal	  5	    1.0
                     closantel	  4	    0.8
                    cilostazol	  2	    0.0
     Chloramphenicol 

In [44]:
model_to_s3(disamb)